In [5]:
import sys
sys.executable

'/home/iruser/miniconda3/bin/python'

In [4]:
# # 데몬 인스턴스 만들기 
# import os
# import json
# from elasticsearch import Elasticsearch, helpers
# from subprocess import Popen, PIPE, STDOUT

# es_server = Popen(['/root/home/elasticsearch-8.8.0/bin/elasticsearch'],
#                   stdout=PIPE, 
#                   stderr=STDOUT,
#                   preexec_fn=lambda: os.setuid(1))

# import time
# time.sleep(60)

PermissionError: [Errno 13] Permission denied: '/root/home/elasticsearch-8.8.0/bin/elasticsearch'

In [10]:
# 데몬 구동 확인 -> password 설정 
!ps -ef | grep elasticsearch

iruser    334920  334797  0 12:18 ?        00:00:02 /home/iruser/elasticsearch-8.8.0/jdk/bin/java -Xms4m -Xmx64m -XX:+UseSerialGC -Dcli.name=server -Dcli.script=./elasticsearch-8.8.0/bin/elasticsearch -Dcli.libs=lib/tools/server-cli -Des.path.home=/home/iruser/elasticsearch-8.8.0 -Des.path.conf=/home/iruser/elasticsearch-8.8.0/config -Des.distribution.type=tar -cp /home/iruser/elasticsearch-8.8.0/lib/*:/home/iruser/elasticsearch-8.8.0/lib/cli-launcher/* org.elasticsearch.launcher.CliToolLauncher
iruser    335025       1  0 12:18 ?        00:00:00 /home/iruser/elasticsearch-8.8.0/modules/x-pack-ml/platform/linux-x86_64/bin/controller
iruser    344816       1 99 14:36 pts/11   00:00:48 /home/iruser/elasticsearch-8.8.0/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.nett

In [11]:
es_username = 'elastic'
es_password = '5+xI3rlxXcfCyNxUJ*qY'

# ElasticSearch Client 생성 
es = Elasticsearch(['https://localhost:9200'], basic_auth=(es_username, es_password), ca_certs="/home/iruser/elasticsearch-8.8.0/config/certs/http_ca.crt")

# ElasticSearch Client 정보 확인 
es.info()

ObjectApiResponse({'name': 'instance-8909', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'r4Ih9u1sTfKbhPDz61kqLQ', 'version': {'number': '8.8.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f', 'build_date': '2023-05-23T17:16:07.179039820Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [12]:
from sentence_transformers import SentenceTransformer

# Sentence Transformer 모델 초기화(한국어 임베딩 생성 가능한 어떤 모델도 가능)
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")   # 이 모델을 사용하여 doc['content']를 임베딩 

# Sentence Transformer를 이용하여 임베딩 생성
def get_embedding(sentences):
    return model.encode(sentences)

# 주어진 문서의 리스트에서 배치 단위로 임베딩 생성
def get_embeddings_in_batches(docs, batch_size=100):
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i: i+batch_size]
        contents = [doc["content"] for doc in batch]
        embeddings = get_embedding(contents)    # doc의 "content" 부분을 임베딩 
        batch_embeddings.extend(embeddings)
        print(f'batch {i}')
    return batch_embeddings

### 인덱스 설정 함수: Sparse Retrieve, Dense Retrieve를 위한 인덱스 생성과 관리
- Dense Retrieve: 인덱스 맵핑 뿐만 아니라 밀집 벡터 필드를 추가해야

In [16]:
# 새로운 index 생성 
def create_es_index(index, settings, mappings):
    if es.indices.exists(index=index):   # 만약 인덱스가 이미 존재한다면 
        es.indices.delete(index=index)   # 새로운 것으로 설정하기 위해 삭제 
    es.indices.create(index=index, settings=settings, mappings=mappings)  # 지정된 settings로 새로운 인덱스 생성 

# 지정된 인덱스 삭제 
def delete_es_index(index):
    es.indices.delete(index=index)
    
# ElasticSearch 헬퍼 함수를 사용하여 대량 인덱싱 수행 
def bulk_add(index, docs):
    actions = [{'_index': index, '_source': doc} for doc in docs]
    return helpers.bulk(es, actions)

### 검색 함수
- Sparse Retrieve: 텍스트 기반 검색 
- Dense Retrieve: 벡터 유사도 기반 검색

In [17]:
# 역색인을 이용한 검색 
def sparse_retrieve(query_str, size):
    query = { "match": {"content": {"query": query_str}}}
    return es.search(index='test', query=query, size=size, sort='_score')
#>> query_str: 검색어
#>> size: 검색 결과 개수 
#>> 'test' 인덱스에서 정의된 쿼리를 실행 -> 검색 결과를 size개 만큼 반환 


# Vector 유사도를 이용한 검색
def dense_retrieve(query_str, size):
    query_embedding = get_embedding([query_str])[0]   # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기 
    knn = {"field": "embeddings",                     # KNN을 사용한 벡터 유사성 검색을 위한 매개변수 설정 
           "query_vector": query_embedding.tolist(),
           "k": size,    # k: 검색할 최근접 이웃의 개수 
           "num_candidates": 100}  # KNN 검색에 사용할 후보 문서 개수 
    return es.search(index='test', knn=knn)           # 지정된 인덱스에서 벡터 유사도 검색 수행 

### 설정

In [19]:
# 색인을 위한 settings 설정 
settings = { "analysis": {"analyzer": {"nori": {"type": "custom",  #>> custom: 사용자 정의 분석기임을 명시 
                                                "tokenizer": "nori_tokenizer",
                                                "decompound_mode": "mixed",  # 복합 명사를 분해하는 방식을 설정. mixed: 복합명사를 분해하되, 분해된 토큰과 원본 토큰을 모두 저장 
                                                "filter": ["nori_posfilter"]}},
                          "filter": {"nori_posfilter":{"type": "nori_part_of_speech",
                                                       "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]}}}  # 제거할 품사 태그를 지정 
            }

# 색인을 위한 mapping 설정(역색인 필드, 임베딩 필드 모두 설정)
mappings = { "properties": {"content": {"type": "text",
                                        "analyzer": "nori"},
                            "embeddings": {"type": "dense_vector",
                                           "dims": 768,
                                           "index": True,
                                           "similarity": "l2_norm"}}  # 벡터 간 유사도 계산 방식: L2 norm 
            }


In [20]:
# 위에서 설정한 내용으로 'test' 인덱스 생성 
create_es_index("test", settings, mappings)

### 임베딩 함수 

In [ ]:
# 문서의 content 필드에 대한 임베딩 생성 
